## Lookuptable experiment

In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=3
!echo $CUDA_VISIBLE_DEVICES

env: CUDA_VISIBLE_DEVICES=3
3


In [2]:
## discovery cluster setup
# CACHE_DIR = "/scratch/lucchetti.f/models/"
# os.environ['TRANSFORMERS_CACHE'] = CACHE_DIR

import os
import sys
sys.path.append('..')

import random
import gc
import torch
import pandas as pd
from model_utils import *
import tqdm as notebook_tqdm
from trace_model import ModelLoader
from pandas import DataFrame
torch.set_grad_enabled(False)
from tqdm import tqdm 
from model_utils import layername
import numpy as np
from extract_data import *
import nltk
nltk.download('words')
from random import sample
from patch import *

import re
check_devs()
# clear_devs()
torch.cuda.is_available()

MODEL_NAME = "/home/arjun/models/starcoderbase-1b/"

/home/franlucc/mechinterp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0 / 84978434048 used for device 0, reserved 0


[nltk_data] Downloading package words to /home/franlucc/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [3]:
lm = ModelLoader(MODEL_NAME)
check_devs()

4615938048 / 84978434048 used for device 0, reserved 4617928704


## synthetic look-up table

In [4]:

prompt = """def func(i : int) -> str:
    if i == 0:
        return "cat"
    elif i == 1:
        return "speed"
    elif i == 3:
        return "city"
    else:
        return "number"
"""

def make_assert(n):
    return f"assert func({n}) == \""

assertions = [
    "assert func(0) == \"",
    "assert func(1) == \"",
    "assert func(2) == \"",
    "assert func(3) == \"",
    "assert func(999) == \"",
    
]

def layernum(lname):
    return int(lname.split(".")[-1])

### Step 1: prune transformer, get earliest layer that successfully completes task

problem: sampling is fast, greedy is slow

In [ ]:
txt, ret_dict = lm.trace_generate(prompt + "\n" + make_assert(99),
                                  max_new_toks=1,
                                  request_logits=[layername(lm.model, l) for l in range(10,30)],
                                  sample_from_topk_logits=None)
out = txt.replace(prompt + "\n" + make_assert(99), "")


In [ ]:
print(out)
ret_dict["logits"]

In [5]:
def trace_lookuptable(max_n, prompt, sample_from=None):
    res = []
    exec(prompt)
    for n in range(0, max_n):
        # for k in range(0, 20):
            txt, ret_dict = lm.trace_generate(prompt + "\n" + make_assert(n),
                                            max_new_toks=1,
                                            request_logits=[layername(lm.model, l) for l in range(10,30)],
                                            sample_from_topk_logits=sample_from)
            out = txt.replace(prompt + "\n" + make_assert(n), "")
            for lname, logits in ret_dict["logits"].items():
                if out in [str(x) for (x,_) in logits]:
                    gold = eval(f"func({n})")
                    res.append(((out == gold), out, gold, n, layernum(lname)))
                    break
    return res

## Try to break lookup table task

make harder by increasing lenght. At what point is transformer unsure what task this is?

In [6]:
def save_to_csv(res, name, header=['prompt_seed','is_correct','completion', 'gold', 'input n', 'layernum']):
    ds = DataFrame.from_dict(res)
    ds.to_csv(f"../results/{name}.csv", header=header)

In [7]:
def make_lookup_table(n, seed=0):
    assert n < 25, "n must be less than 25"
    random.seed(seed)
    nums = sample(list(range(n)), n)
    words = sample(list(range(n+1)), n+1)
    s = f"def func(i : int) -> str:\n\tif i == {nums[0]}:\n\t\treturn \"{chr(97+words[0])}\"\n"
    for i,k in enumerate(nums[1:]):
        s += f"\telif i == {k}:\n\t\treturn \"{chr(97+words[i+1])}\"\n"
    s += f"\telse:\n\t\treturn \"{chr(97+words[-1])}\""
    return s.replace("\t","    ")

In [8]:
def trace_false_task(max_n, prompt, seed, len_table=12):
    res = []
    exec(prompt)
    
    for n in range(0, max_n):
        # for k in range(0, 20):
            txt, ret_dict = lm.trace_generate(prompt + "\n" + make_assert(n),
                                            max_new_toks=1,
                                            request_logits=[layername(lm.model, l) for l in range(10,20)],
                                            do_greedy_decoding=True)
            out = txt.replace(prompt + "\n" + make_assert(n), "")
            for lname, logits in ret_dict["logits"].items():
                if out in [str(x) for (x,_) in logits]:
                    gold = eval(f"func({n})")
                    def first_or_last_match(out, i):
                        exec(prompt)
                        first_digit = eval(f"func({str(i)[0]})")
                        last_digit = eval(f"func({str(i)[-1]})")
                        if first_digit == out:
                            return "Match_First_Digit_False_Task"
                        elif last_digit == out:
                            return "Match_Last_Digit_False_Task"
                        else:
                            return "Neither_False_Task"
                    is_correct = (out == gold)
                    first_or_last_match_str = first_or_last_match(out, n) if not is_correct else "NA"
                    res.append((len_table, seed, is_correct, first_or_last_match_str, out, gold, n, layernum(lname)))
                    break
    return res

In [9]:
import json
def exp_false_task(seed_range, lengths_table=[3,6,9,12], seed_start=0):
    fp = open(f"../results/log.json", "w")
    tot_res = []
    for len_table in tqdm(lengths_table):
        for seed in range(seed_start, seed_range):
            prompt = make_lookup_table(len_table, seed=seed)
            res = trace_false_task(50, prompt, seed, len_table=len_table)
            json.dump(res, fp)
            tot_res += res
    fp.close()
    return tot_res


In [10]:
tot_res = exp_false_task(2)
save_to_csv(tot_res, "1ftb_full_lookuptable_task0", header =["table_len","seed","is_correct","first_or_last_match","out","gold","func(n)","earliest_layer"])

100%|██████████| 4/4 [16:27<00:00, 246.77s/it]


In [11]:
tot_res = exp_false_task(20, seed_start=2)
save_to_csv(tot_res, "1b_full_lookuptable_task1", header =["table_len","seed","is_correct","first_or_last_match","out","gold","func(n)","earliest_layer"])

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [20:00<?, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 tot_res = exp_false_task(20, seed_start=2)                                                   │
│   2 save_to_csv(tot_res, "1b_full_lookuptable_task1", header =["table_len","seed","is_correc     │
│   3                                                                                              │
│                                                                                                  │
│ in exp_false_task:8                                                                              │
│                                                                                                  │
│    5 │   for len_table in tqdm(lengths_table):                                                   │
│    6 │   │   for seed in range(seed_start, seed_range):                                          │
│    7 │   │   │   prompt = make_lookup_table(len_table, seed=seed)                                │
│ ❱  8 │   │   │   res = trace_false_task(50, prompt, seed, len_table=len_table)                   │
│    9 │   │   │   json.dump(res, fp)                                                              │
│   10 │   │   │   tot_res += res                                                                  │
│   11 │   fp.close()                                                                              │
│                                                                                                  │
│ in trace_false_task:7                                                                            │
│                                                                                                  │
│    4 │                                                                                           │
│    5 │   for n in range(0, max_n):                                                               │
│    6 │   │   # for k in range(0, 20):                                                            │
│ ❱  7 │   │   │   txt, ret_dict = lm.trace_generate(prompt + "\n" + make_assert(n),               │
│    8 │   │   │   │   │   │   │   │   │   │   │   max_new_toks=1,                                 │
│    9 │   │   │   │   │   │   │   │   │   │   │   request_logits=[layername(lm.model, l) for l    │
│   10 │   │   │   │   │   │   │   │   │   │   │   do_greedy_decoding=True)                        │
│                                                                                                  │
│ /home/franlucc/codetrace/notebooks/../trace_model.py:215 in trace_generate                       │
│                                                                                                  │
│   212 │   │   │   │                                                                              │
│   213 │   │   │   │   # collect tokens generated from final layer                                │
│   214 │   │   │   │   for i in range(input_ids.size(0)):                                         │
│ ❱ 215 │   │   │   │   │   generated_tokens[i].append(                                            │
│   216 │   │   │   │   │   │   [                                                                  │
│   217 │   │   │   │   │   │   │   {"token": self.tokenizer.decode(t), "id": t.item(), "p": sof   │
│   218 │   │   │   │   │   │   │   for t in topk_logits[i]                                        │
│                                                                                                  │
│ /home/franlucc/codetrace/notebooks/../trace_model.py:216 in <listcomp>                           │
│                                                                                                  │
│   213 │   │   │   │   # collect tokens generated from final

In [ ]:
tot_res = exp_false_task(20)
save_to_csv(tot_res, "1b_lookup_table_false_task_small0")

## Find the attn heads responsible

layer 18

In [7]:
heads_to_noise = [(h, 18) for h in range(48)]
## noise create
uniform_noise = False
noise = 0.1
replace=True
rs = numpy.random.RandomState(1)  # For reproducibility, use pseudorandom noise
if uniform_noise:
    prng = lambda *shape: rs.uniform(-1, 1, shape)
else:
    prng = lambda *shape: rs.randn(*shape)
if isinstance(noise, float):
    noise_fn = lambda x: noise * x
else:
    noise_fn = noise
    
heads_to_noise = [(head, layername(lm.model, layer)) for (head, layer) in heads_to_noise]
head_dim = lm.model.config.n_embd // lm.model.config.n_head
def patch_rep(x, layer): # x is the output of the attn forward method (layer), layer is layer num
    ## x is tuple (attn_weights, present[key value tracker for MQA], opt(attn_weights))
    if layer in [layer for (_, layer) in heads_to_noise]:
        for head in [head for (head, layer_) in heads_to_noise if layer_ == layer]:
            print("patching (layer, head): ", layer, head)
            ## noise at the corresponding heads
            start = head_dim * head
            end = start + head_dim
            noise_data = noise_fn(
                    torch.from_numpy(prng(x[0].shape[0], x[0].shape[1], head_dim))
                ).to(x[0].device)
            if replace:
                x[0][:,:,start:end] = noise_data
            else:
                x[0][:,:,start:end] += noise_data
    return x


In [8]:
td = TraceDict(lm.model, [layername(lm.model, i) for i in range(1, 40)], edit_output_attn=patch_rep)

In [19]:
import json
with open("../results/lookup_table_false_task2.csv", "r") as f:
    df = pd.read_csv(f)

In [42]:

for index, row in df.iterrows():
    txt = noise_attn_heads(lm, 
                        make_lookup_table(12, seed=int(row["prompt_seed"])) + "\n" + make_assert(int(row["input n"])), 
                        heads_to_noise=[(h, 18) for h in range(48)]+[(h, 17) for h in range(48)])
    if txt[0] == row["gold"]:
        print("True ", str(txt)+str(index)+str(row["gold"])+str(row["completion"]))
    else:
        print("False ", str(txt)+str(index)+str(row["gold"])+str(row["completion"]))

True  ['d']0dl
True  ['d']1dj
True  ['d']2dj
True  ['d']3dj
True  ['d']4dj
False  ['e']5de
True  ['d']6dj
True  ['d']7dj
True  ['d']8dj
True  ['k']9kh
True  ['k']10kh
True  ['k']11kh
True  ['k']12kh
True  ['k']13kh
False  ['j']14kj
False  ['j']15kj
True  ['k']16kj
False  ['j']17kj
False  ['j']18kj
True  ['k']19kj
False  ['j']20kj
False  ['j']21kj
False  ['a']22ba
False  ['a']23ba
False  ['a']24bl
False  ['a']25ba
True  ['b']26bl
False  ['a']27bl
False  ['a']28bl
False  ['a']29bi
True  ['b']30bi
True  ['b']31bl
True  ['b']32bi
False  ['i']33bi
False  ['a']34bi
False  ['a']35bi
False  ['a']36ga
False  ['e']37ge
False  ['e']38ge
False  ['a']39gl
False  ['a']40gl
False  ['l']41gl
False  ['e']42gl
False  ['e']43gl
False  ['a']44gl
False  ['a']45ga
True  ['k']46km
False  ['m']47km
False  ['m']48km
False  ['m']49km
False  ['m']50km
False  ['d']51kd
False  ['d']52kd
False  ['d']53kd
False  ['d']54kd
False  ['d']55ki
True  ['k']56ki
False  ['d']57ki
True  ['k']58ki
False  ['e']59ki
True  ['k']6

In [14]:
lm.generate(make_lookup_table(12, seed=0) + "\n" + make_assert(48),max_new_tokens=1, return_only_generated=True)

'j'

In [16]:
txt, ret = lm.trace_generate(make_lookup_table(12, seed=0) + "\n" + make_assert(48),
                  max_new_toks=1, 
                  do_greedy_decoding=True)

In [17]:
txt

'def func(i : int) -> str:\n    if i == 6:\n        return "e"\n    elif i == 11:\n        return "c"\n    elif i == 0:\n        return "b"\n    elif i == 4:\n        return "j"\n    elif i == 7:\n        return "m"\n    elif i == 3:\n        return "l"\n    elif i == 2:\n        return "h"\n    elif i == 10:\n        return "a"\n    elif i == 5:\n        return "f"\n    elif i == 8:\n        return "g"\n    elif i == 9:\n        return "k"\n    elif i == 1:\n        return "i"\n    else:\n        return "d"\nassert func(48) == "j'

In [13]:
print(make_lookup_table(12, seed=0))

def func(i : int) -> str:
    if i == 6:
        return "e"
    elif i == 11:
        return "c"
    elif i == 0:
        return "b"
    elif i == 4:
        return "j"
    elif i == 7:
        return "m"
    elif i == 3:
        return "l"
    elif i == 2:
        return "h"
    elif i == 10:
        return "a"
    elif i == 5:
        return "f"
    elif i == 8:
        return "g"
    elif i == 9:
        return "k"
    elif i == 1:
        return "i"
    else:
        return "d"
